In [22]:
import tensorflow as tf
import scipy as sp
from scipy import ndimage
import numpy as np
import os

In [28]:
# The inputs and outptus are the maps downloaded from https://www.nnvl.noaa.gov/view/globaldata.html
# The inputs to the model are Land Surface Temperature, Moisture and Longwave Energy. 
# The output is the average precipitation map. 
# All maps are in the average weekly format. The idea is to find correlation between the input maps and 
# the output. 

input_labels = ['fire', 'frac', 'land', 'moisture', 'ndvi', 'longwave_energy'];
files = ['fire_data.txt', 'frac_data.txt', './land_data.txt', './mois_data.txt', './ndvi_data.txt', './olwr_data.txt']
input_dict = {}

xdim = 256
ydim = 512
in_chan = 6
out_chan = 1

# Process the input : input_metrics will contain a 2048 x 4096 image with 9 channels (corresponding to 3 inputs)

for label, path in zip(input_labels, files):
    
    input_dict[label] = [];
    input_dict[label] = (np.loadtxt(path,delimiter=',').reshape(-1,xdim,ydim,1))[:35,:,:,:]
    print(label, path,input_dict[label].shape)

input_metrics = input_dict[input_labels[0]];
for label in (input_labels[1:]):
    input_metrics = np.concatenate((input_metrics, input_dict[label]), axis=3);

# Process the output : output_metric will contain a 256 x 4096 image with 3 channels
path = './rain_data.txt'
output_metric = (np.loadtxt(path, delimiter=',').reshape(-1,xdim,ydim,1)[:35,:,:,:])
output_metric = np.array(output_metric);
samples = input_metrics.shape[0];

print(input_metrics.shape)
print(output_metric.shape)


fire fire_data.txt (35, 256, 512, 1)
frac frac_data.txt (35, 256, 512, 1)
land ./land_data.txt (35, 256, 512, 1)
moisture ./mois_data.txt (35, 256, 512, 1)
ndvi ./ndvi_data.txt (35, 256, 512, 1)
longwave_energy ./olwr_data.txt (35, 256, 512, 1)
(35, 256, 512, 6)
(35, 256, 512, 1)


In [30]:
# Function to obtain the next batch based on the input size and the batch size
def next_batch(indices, i):
    
    ind0, ind1 = i*batch_size, np.minimum((i+1)*batch_size, samples)
            
    return input_metrics[indices[ind0:ind1], :, :, :], output_metric[indices[ind0:ind1], :, :, :]

In [34]:
# Building a simple CNN model that looks like an auto-encoder. This is the section to change for a new model.

def conv_net(x):
    
    x = tf.reshape(x, shape=[-1, xdim, ydim, in_chan], name='reshape_x');
    x = tf.cast(x, tf.float32) 
    
    # Encoder
    # 2048x4096x9 -> 1024x2048x32
    conv1 = tf.nn.relu(tf.contrib.layers.conv2d(x, 32, [8, 8], stride=2, padding='SAME'))
    # 1024x2048x32 -> 512x1024x16
    conv2 = tf.nn.relu(tf.contrib.layers.conv2d(conv1, 16, [8, 8], stride=2, padding='SAME'))
    # 512x1024x16 -> 256x512x8
    conv3 = tf.nn.relu(tf.contrib.layers.conv2d(conv2, 8, [8, 8], stride=2, padding='SAME'))
    
    # Decoder
    # 256x512x8 -> 512x1024x16
    conv4 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv3, 16, [8, 8], stride=2, padding='SAME'))
    # 512x1024x16 -> 1024x2048x32
    conv5 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv4, 32, [8, 8], stride=2, padding='SAME'))
    # 1024x2048x32 -> 2048x4096x3
    conv6 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv5, out_chan, [8, 8], stride=2, padding='SAME'))
    
    return conv6;

Y = tf.placeholder(tf.float32, shape=(None,xdim,ydim,out_chan));
X = tf.placeholder(tf.float32, shape=[None,xdim,ydim,in_chan]);

optimizer = tf.train.AdamOptimizer(learning_rate=0.1);

out = conv_net(X);
loss_op = tf.reduce_sum(tf.multiply(Y-out,Y-out));
train_op = optimizer.minimize(loss_op);

In [41]:
num_epochs = 100;
batch_size = 8;
num_batches = int(np.ceil(float(samples) / batch_size));
print(num_batches)

with tf.Session() as sess:
    
    # Run the initializer
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        indices = np.random.permutation(samples);
    
        for i in range(num_batches):
            x_train, y_train = next_batch(indices, i);
            sess.run([train_op], feed_dict={X: x_train, Y: y_train});            
        
        loss = sess.run([loss_op], feed_dict={X: x_train, Y: y_train});
        print(epoch, loss);

4
0 [3.0249659e+09]
1 [2.0025527e+09]
2 [2.5506522e+09]
3 [2.2017997e+09]
4 [3.0296184e+09]
5 [2.8798351e+09]
6 [2.723893e+09]
7 [2.0725866e+09]
8 [2.6847706e+09]
9 [2.7234849e+09]
10 [3.2241866e+09]
11 [2.2098537e+09]
12 [3.6641385e+09]
13 [2.6662845e+09]
14 [2.8257935e+09]
15 [2.3834993e+09]
16 [2.2946624e+09]
17 [2.6696645e+09]
18 [2.4607155e+09]
19 [2.4007247e+09]
20 [3.0113628e+09]
21 [2.6558577e+09]
22 [2.8963482e+09]
23 [2.9861304e+09]
24 [2.6723996e+09]
25 [2.9446956e+09]
26 [2.4405875e+09]
27 [2.4430456e+09]
28 [2.3430372e+09]
29 [2.603906e+09]
30 [2.6857178e+09]
31 [3.1590057e+09]
32 [2.9702428e+09]
33 [3.023498e+09]
34 [2.3405775e+09]
35 [2.8242012e+09]
36 [2.4794726e+09]
37 [2.596928e+09]
38 [2.8512817e+09]
39 [3.0350751e+09]
40 [2.2044918e+09]
41 [2.1067218e+09]
42 [1.9339328e+09]
43 [2.6844623e+09]
44 [2.4665981e+09]
45 [2.7954596e+09]
46 [2.2914624e+09]
47 [2.2221412e+09]
48 [2.4618276e+09]
49 [2.4642634e+09]
50 [2.2503355e+09]
51 [2.9391411e+09]
52 [2.5056753e+09]
53 [2